# Autonomous Trading Agents - With 0xMONK from FereAI

<a target="_blank" href="https://colab.research.google.com/github/fere-ai/agentic-examples/blob/main/python/trading-agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook demonstrates the ease of building autonomous trading agents. It's super easy using the `0xMONK` APIs from FereAI.

- [Developer Docs](https://docs.fereai.xyz/docs/api/api-introduction)
- [Get Your API Key](https://docs.google.com/forms/d/e/1FAIpQLScFAWfT5u2kwKx8tQkkL9AfdP8NIlNIn6tXJUQcRipZEUZokA/viewform)
- [#dev-help on our Discord](https://discord.com/invite/3fsm5XJNW8)

In [ ]:
import requests
import json
from pprint import pprint
from tabulate import tabulate
from uuid import  UUID

In [ ]:
fere_api_key = "1234" # @param {type: "string"}
fere_user_id = "d9a486fa-8dd0-4593-b2f6-9443d3a0abd6" # @param {type: "string"}

The main class `DiscipleClient` has all the methods needed to interact with Disciple APIs

In [ ]:
class DiscipleClient:
    base_url = "https://api.fereai.xyz/ta/"
    # base_url = "http://localhost:8000/"

    def __init__(self, api_key, user_id):
        """
        Initializes the FereAI class with an API key.

        Args:
            api_key (str): The API key for authenticating requests.
            user_id (str): The user ID for identifying the user.
        """
        self.api_key = api_key
        self.user_id = user_id

    @property
    def api_headers(self):
        """
        Generates headers for API requests.

        Returns:
            dict: The headers including the Authorization key.
        """
        return {
            "X-FRIDAY-KEY": self.api_key,
            "Content-Type": "application/json",
            "X-FERE-USERID": self.user_id,
        }

    def create_agent(
      self,
      fere_user_id,
      name,
      description,
      persona,
      data_source,
      decision_prompt_pool,
      decision_prompt_portfolio,
      twitter_username=None,
      fc_username=None,
      simulation=False,
      simulation_initial_usd=1000,
      simulation_initial_native=1,
      max_investment_per_session=0.2,
      stop_loss=0.6,
      trailing_stop_loss=None,
      take_profit=0.5,
      chain='base',
      mode='MANUAL',
      ):
      """
      Creates a new trading agent.

      Args:
          fere_user_id (str): The user ID.
          name (str): The name of the agent.
          description (str): A description of the agent.
          persona (str): The persona of the agent.
          data_source (str): The data source for the agent.
          decision_prompt_pool (str): The decision prompt pool.
          decision_prompt_portfolio (str): The decision prompt portfolio.
          twitter_username (str, optional): Twitter username.
          fc_username (str, optional): FC username.
          simulation (bool, optional): Whether to run in simulation mode.
          simulation_initial_usd (int, optional): Initial USD for simulation.
          simulation_initial_native (int, optional): Initial native currency for simulation.
          max_investment_per_session (float, optional): Max investment per session.
          stop_loss (float, optional): Stop loss percentage.
          trailing_stop_loss (float, optional): Trailing stop loss percentage.
          take_profit (float, optional): Take profit percentage.
          chain (str, optional): Blockchain chain.

      Returns:
          dict or None: The created agent data or None on failure.
      """
      url = self.base_url + "agent/"
      payload = json.dumps({
          "user_id": fere_user_id,
          "name": name,
          "description": description,
          "persona": persona,
          "chain": chain,
          "data_source": data_source,
          "decision_prompt_pool": decision_prompt_pool,
          "decision_prompt_portfolio": decision_prompt_portfolio,
          "twitter_username": twitter_username,
          "fc_username": fc_username,
          "dry_run": simulation,
          "dry_run_initial_usd": simulation_initial_usd,
          "dry_run_initial_native": simulation_initial_native,
          "max_investment_per_session": max_investment_per_session,
          "stop_loss": stop_loss,
          "trailing_stop_loss": trailing_stop_loss,
          "take_profit": take_profit,
          "mode": mode,
      })

      response = requests.put(url, headers=self.api_headers, data=payload, timeout=10)

      if response.status_code == 200:
          data = response.json()
          pprint("Agent created")
          pprint(data)
          return data
      else:
          print(response.text)
          return None

    def get_agent(self, agent_id: UUID | str) -> dict:
        """
        Fetches an agent by ID.

        Args:
            agent_id (str): The agent ID.

        Returns:
            dict or None: The agent data or None on failure.
        """
        url = f"{self.base_url}agent/{agent_id}"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def fetch_disciples(self, fere_user_id):
        """
        Fetches disciples for a user ID.

        Args:
            fere_user_id (str): The user ID.

        Returns:
            list: A list of disciples or an empty list on failure.
        """
        url = f"{self.base_url}agent/{fere_user_id}/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            data = response.json()
            print(f"Disciples Created: {len(data)}\n")
            for disciple in data:
                print(f"Name: {disciple['name']}")
                print(f"ID: {disciple['id']}\n")
            return data
        else:
            print(f"Failed to fetch disciples: {response.text}")
            return []

    def get_portfolio(self, disciple_agent_id):
        """
        Fetches portfolio for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The portfolio data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/portfolio/"
        response = requests.get(url, headers=self.api_headers, timeout=60)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_holdings(self, disciple_agent_id):
        """
        Fetches holdings for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            list or None: The holdings data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/holdings/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(response.text)
            return None

    def get_trades(self, disciple_agent_id):
        """
        Fetches trades for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The trades data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/trades/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            pprint(response.text)
            return None

    def get_trade_recommendations(self, disciple_agent_id):
        """
        Fetches trade recommendations for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The trade recommendations data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/decisions/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_optimal_gains(self, disciple_agent_id):
        """
        Fetches optimal gains for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The optimal gains data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/calls/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_decisions(self, disciple_id: UUID):
        """Gets the decisions of the agent.
        """
        url = f"{self.base_url}agent/{disciple_id}/decisions/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def update_agent(self, disciple_id, **kwargs):
        """
        Updates an agent.

        Args:
            disciple_id (str): The agent ID.
            **kwargs: Parameters for updating the agent.

        Returns:
            dict or None: The updated agent data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_id}/"
        payload = json.dumps(kwargs)
        response = requests.patch(url, headers=self.api_headers, data=payload, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def schedule_sell(self, disciple_id, holding_id, quantity):
      """
      Schedules a sell for a holding.

      Args:
          disciple_id (str): The disciple ID.
          holding_id (str): The holding ID.
          quantity (str): The quantity to sell.

      Returns:
          dict or None: The response data or None on failure.
      """
      url = f"{self.base_url}/agent/{disciple_id}/sell/f{holding_id}/f{quantity}/"
      payload = {}

      response = requests.request("POST", url, headers=self.api_headers, data=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        print(response.text)

    def schedule_buy(self, disciple_id: UUID, ca: str, quantity: int, slippage: float = 1.0):
      """
      Schedules a buy for a symbol.

      Args:
          disciple_id (str): The disciple ID.
          symbol (str): The symbol to buy.
          quantity (str): The quantity to buy.

      Returns:
          dict or None: The response data or None on failure.
      """
      url = f"{self.base_url}/agent/{disciple_id}/buy/"
      payload = {
          "ca": ca,
          "quantity": quantity,
          "slippage": slippage,
      }
      print(payload)

      response = requests.request("POST", url, headers=self.api_headers, json=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        print(response.text)

    def check_task_status(self, task_id):
      """
      Checks the status of a task.

      Args:
          task_id (str): The task ID.

      Returns:
          dict or None: The task status data or None on failure.
      """
      url = f"{self.base_url}/task/status/{task_id}/"

      payload = {}

      response = requests.request("GET", url, headers=self.api_headers, data=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        print(response.text)

    def delete_disciple(self, disciple_agent_id):
        """
        Deletes a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The response data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/"
        response = requests.delete(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None


## 0. Create a Disciple Instance

In [ ]:
disciple = DiscipleClient(fere_api_key, fere_user_id)


## 1. Create an Agent

Instructions Examples: https://docs.fereai.xyz/docs/product/0xMONK/#coin-decision-instructions


In [ ]:
name = "Disciple latest" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "latest" # @param ["trending", "latest", "virtuals_s"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = 0.2 # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = True # @param {type: "boolean"}
simulation_initial_usd = None # @param {type: "number"}
simulation_initial_native = 1 # @param {type: "number"}
chain = "solana" # @param ["base", "solana", "arbitrum"]
mode = "MANUAL" # @param ["MANUAL", "COPY_TRADE", "AUTONOMOUS"]

In [ ]:
agent = disciple.create_agent(fere_user_id, name,description,persona,data_source,
             decision_prompt_pool, decision_prompt_portfolio, twitter_username,
             fc_username, simulation, simulation_initial_usd, simulation_initial_native,
             max_investment_per_session, stop_loss, trailing_stop_loss,
             take_profit, chain, mode)

pprint(agent)

## 2. Get Disciple Agents from a given User

In [ ]:
pprint(disciple.fetch_disciples(fere_user_id))

## 3. Get portfolio

In [ ]:
disciple_agent_id = "17491a7f-3049-4346-abc8-0d8cba1a55af" # @param {type: "string"}


In [ ]:
portfolio = disciple.get_portfolio(disciple_agent_id)

curr_amount = portfolio['curr_realised_usd'] + portfolio['curr_unrealised_usd']
profit = curr_amount - portfolio['start_usd']
profit_percent = (profit / portfolio['start_usd']) * 100

pprint(portfolio)
pprint(f"Profit %: {profit_percent}")

## 4. Get Holdings

In [ ]:
holdings = disciple.get_holdings(disciple_agent_id)

print(f"Total holdings: {len(holdings)}")

_h = []
for h in holdings:
  _h.append([h['base_address'], h['pool_name'], h['tokens_bought']])

headers = ["Base Address", "Pool Name", "Tokens Bought"]
print(tabulate(_h, headers=headers))

## 5. Get Trades


In [ ]:

trades = disciple.get_trades(disciple_agent_id)
print(trades)

_t = []
for t in trades:
  _t.append([
    t['base_address'],
    t['pool_name'],
    'buy' if t['decision'] == 1 else 'sell',
    t['in_amount'],
    t['out_amount'],
    t['price_usd'],
    t['created_at'],
    t['txn']])

headers =  ['Base Address', 'Pool Name', 'Decision', 'In Amount', 'Out Amount', 'Price USD', 'Created At', 'Txn']

print(f"Total Trades: {len(trades)}")

print(tabulate(_t, headers=headers))

## 6. Get Optimal gains

In [ ]:
disciple.get_optimal_gains(disciple_agent_id)

## 7. Sell a holding manually

In [ ]:
holding_id = "" # @param {type: "string"}
quantity = "all" # @param {type: "string"}

A sell is executed as a background task. Upon calling the sell API, it returns a task ID. This task ID will be used to check the task status.

In [ ]:

task = disciple.schedule_sell(disciple_agent_id, holding_id, quantity)

pprint(task)

Now once we have the task ID, we can check the status of the task.

In [ ]:
status = disciple.check_task_status(task['task_id'])

print(status)

Depending on the status, show the updates to user.

## 8. Buy a coin manually

In [ ]:
solana_ca = "6uGrKeFiTccSi4JdTQnfu43bn4p5A8qnSqSbABampump" # @param {type: "string"}
base_ca = "0x23dD3Ce6161422622E773E13dAC2781C7f990D45" # @param {type: "string"}
arb_ca = "0x37a645648dF29205C6261289983FB04ECD70b4B3" # @param {type: "string"}

ca_to_buy = solana_ca

In [ ]:
task_ = disciple.schedule_buy(disciple_agent_id, ca_to_buy, 100, 1)

In [ ]:
status = disciple.check_task_status(task_['task_id'])

print(status)

## 9. Get the decisions of the Agent

In [ ]:
response = disciple.get_decisions(disciple_agent_id)

pprint(response)

# Disciple Lifecycle APIs

## 10. Update a Disciple

In [ ]:
disciple_id = "" # @param {type: "string"}
name = "Disciple 420" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "trending" # @param ["trending", "latest"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = 0.2 # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = True # @param {type: "boolean"}
simulation_initial_usd = 1000 # @param {type: "number"}

In [ ]:
# Set the agent to OnChain Mode
disciple_id = "bd504501-5235-486e-ad78-6906601d1ca6" # @param {type: "string"}

updates = {
    "dry_run": False,
}

disciple.update_agent(disciple_id, **updates)

### 10.1. Pause an agent

In [ ]:

disciple_id = "789aa167-3b2a-4a0f-8734-c82f2d93e22a" # @param {type: "string"}

updates = {
    "is_active": False,
}

disciple.update_agent(disciple_id, **updates)

## 11. Delete the Agent

In [ ]:
res = disciple.delete_disciple(disciple_id)

pprint(res)

## 11. Get an agent details

In [ ]:
res = disciple.get_agent("789aa167-3b2a-4a0f-8734-c82f2d93e22a")
pprint(res)